In [ ]:
# =========================================
# 0️⃣ Install Dependencies
# =========================================
!pip install rdkit py3Dmol pandas numpy scikit-learn selfies --quiet

# =========================================
# 1️⃣ Imports
# =========================================
from rdkit import Chem, RDLogger
from rdkit.Chem import Descriptors, Lipinski, Crippen, Draw, AllChem
from rdkit.Chem.Scaffolds import MurckoScaffold
from rdkit.DataStructs import TanimotoSimilarity
from rdkit.Chem import rdMolDescriptors
import py3Dmol
import pandas as pd
import numpy as np
import selfies as sf
from sklearn.ensemble import RandomForestRegressor
from IPython.display import display, clear_output

RDLogger.DisableLog('rdApp.*')  # suppress RDKit warnings

# =========================================
# 2️⃣ SMILES Validation and Cleaning
# =========================================
def clean_smiles(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None: return None
    smiles_clean = Chem.MolToSmiles(mol, isomericSmiles=False)
    mol_clean = Chem.MolFromSmiles(smiles_clean)
    return smiles_clean if mol_clean else None

def is_valid_smiles(smiles):
    return Chem.MolFromSmiles(smiles) is not None

# =========================================
# 3️⃣ Molecular Descriptors
# =========================================
def compute_features(smiles_list):
    results = []
    for s in smiles_list:
        mol = Chem.MolFromSmiles(s)
        if mol:
            descriptors = [
                Descriptors.MolWt(mol),
                Crippen.MolLogP(mol),
                Lipinski.NumHDonors(mol),
                Lipinski.NumHAcceptors(mol),
                Lipinski.NumRotatableBonds(mol),
                rdMolDescriptors.CalcTPSA(mol)
            ]
            results.append(descriptors)
        else:
            results.append([None]*6)
    df = pd.DataFrame(results, columns=['MolWt','MolLogP','HBD','HBA','RotBonds','TPSA'])
    df['SMILES'] = smiles_list
    return df

# =========================================
# 4️⃣ Dummy ML Models for Scoring
# =========================================
def train_dummy_models(df_features):
    df_features['Solubility'] = np.random.rand(len(df_features))
    df_features['Toxicity'] = np.random.rand(len(df_features))
    X = df_features[['MolWt','MolLogP','HBD','HBA','RotBonds','TPSA']]
    y_sol = df_features['Solubility']
    y_tox = df_features['Toxicity']
    model_sol = RandomForestRegressor(n_estimators=100, random_state=42)
    model_sol.fit(X, y_sol)
    model_tox = RandomForestRegressor(n_estimators=100, random_state=42)
    model_tox.fit(X, y_tox)
    return model_sol, model_tox

# =========================================
# 5️⃣ Molecule Fingerprint
# =========================================
def mol_fingerprint(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None: return None
    return rdMolDescriptors.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048)

# =========================================
# 6️⃣ Generate Similarity-Guided Variants
# =========================================
def generate_variants_similarity(smiles_input, n_variants=30, similarity_threshold=0.5):
    cleaned = clean_smiles(smiles_input)
    if cleaned is None:
        print(f"❌ Invalid SMILES: {smiles_input}")
        return []

    base_mol = Chem.MolFromSmiles(cleaned)
    if base_mol is None: return []

    base_fp = mol_fingerprint(cleaned)
    if base_fp is None: return []

    base_scaffold = MurckoScaffold.GetScaffoldForMol(base_mol)
    variants = set()
    max_attempts = n_variants*20
    attempts = 0

    while len(variants) < n_variants and attempts < max_attempts:
        mol = Chem.MolFromSmiles(cleaned)
        rw_mol = Chem.RWMol(mol)
        if rw_mol.GetNumAtoms() == 0: break

        # Random atom mutation
        atom_idx = np.random.randint(0, rw_mol.GetNumAtoms())
        atom = rw_mol.GetAtomWithIdx(atom_idx)
        atom.SetAtomicNum(int(np.random.choice([6,7,8,9,17,35])))

        try:
            new_smiles = Chem.MolToSmiles(rw_mol, isomericSmiles=False)
            if not is_valid_smiles(new_smiles):
                attempts += 1
                continue

            new_fp = mol_fingerprint(new_smiles)
            sim = TanimotoSimilarity(base_fp, new_fp)
            new_mol = Chem.MolFromSmiles(new_smiles)
            new_scaffold = MurckoScaffold.GetScaffoldForMol(new_mol)
            if sim >= similarity_threshold and Chem.MolToSmiles(new_scaffold) == Chem.MolToSmiles(base_scaffold):
                variants.add(new_smiles)
        except:
            pass
        attempts += 1

    return list(variants)

# =========================================
# 7️⃣ Evaluate Molecules
# =========================================
def evaluate_molecules(smiles_list, model_sol, model_tox):
    df = compute_features(smiles_list)
    X = df[['MolWt','MolLogP','HBD','HBA','RotBonds','TPSA']]
    df['Pred_Solubility'] = model_sol.predict(X)
    df['Pred_Toxicity'] = model_tox.predict(X)
    df['Lab_Feasibility'] = np.random.rand(len(df))
    df['Composite_Score'] = (
        df['Pred_Solubility']*0.3 +
        df['Pred_Toxicity']*0.3 +
        df['Lab_Feasibility']*0.4
    )
    return df.sort_values('Composite_Score', ascending=False)

# =========================================
# 8️⃣ 3D Structures
# =========================================
def generate_3d_structure(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        mol = Chem.AddHs(mol)
        AllChem.EmbedMolecule(mol, randomSeed=42)
        AllChem.MMFFOptimizeMolecule(mol, maxIters=500)
        return mol
    return None

def show_3d_structure(mol):
    block = Chem.MolToMolBlock(mol)
    viewer = py3Dmol.view(width=600, height=400)
    viewer.addModel(block, "mol")
    viewer.setStyle({'stick': {}})
    viewer.setBackgroundColor('white')
    viewer.zoomTo()
    return viewer

# =========================================
# 9️⃣ Main Explorer
# =========================================
def explore_molecules(smiles_list, n_variants=30, export_csv=True):
    clear_output()
    all_variants = []
    for smiles_input in smiles_list:
        print(f"\nInput SMILES: {smiles_input}")
        variants = generate_variants_similarity(smiles_input, n_variants=n_variants)
        print(f"{len(variants)} valid 2D variants generated.")
        all_variants.extend([smiles_input]+variants)

    if len(all_variants) == 0:
        print("❌ No valid variants. Try simpler SMILES.")
        return

    df_features = compute_features(all_variants)
    model_sol, model_tox = train_dummy_models(df_features)
    df_eval = evaluate_molecules(all_variants, model_sol, model_tox)
    display(df_eval.head(10))

    # 3D for top 3 molecules
    top3 = df_eval.head(3)
    for i, row in top3.iterrows():
        print(f"\n3D Structure for {row['SMILES']}")
        mol3d = generate_3d_structure(row['SMILES'])
        viewer = show_3d_structure(mol3d)
        viewer.show()

    if export_csv:
        filename = "top_molecules_export.csv"
        df_eval.to_csv(filename, index=False)
        print(f"\n✅ All variants exported to {filename}")

# =========================================
# 🔟 Run Explorer
# =========================================
smiles_input_raw = input("Enter SMILES (comma separated, e.g., CCCCCCCC, CCC, CC(=O)O): ")
smiles_list = [s.strip() for s in smiles_input_raw.split(",") if s.strip()]
explore_molecules(smiles_list, n_variants=30, export_csv=True)



